In [ ]:
from flask import Flask
from flask import jsonify
from flask import request
from flask_cors import CORS, cross_origin
from collections import OrderedDict
from werkzeug.serving import run_simple
import pymysql
import datetime
import time
import json 
app = Flask(__name__)
app.config['JSON_SORT_KEYS'] = False
CORS(app)

@app.route("/getStandandQuestion", methods=['GET'])
def getStandandQuestion():
    db = request.args.get('db')
    conn = pymysql.connect( host = '10.55.52.98' ,port=33060 ,  user = 'root' ,  passwd = "1234" ,  db = db )     
    cur = conn.cursor() 
        
    returnData = OrderedDict();
     
    cur.execute('SELECT Answer_NO,System,Question,category_main,category_sub,Answer,Keyword,SOP_chapter,SOP_No,URL,Enabled,Creator,Create_Date from '+db+'_standand_question_new order by Answer_NO')
    for r in cur :
        tmp = OrderedDict()
        Answer_NO = r[0]
        tmp['System'] = r[1]
        tmp['Question'] = r[2]
        tmp['category_main'] = r[3]
        tmp['category_sub'] = r[4]
        tmp['Answer'] = r[5]
        tmp['Keyword'] = r[6]
        tmp['SOP_chapter'] = r[7]
        tmp['SOP_No'] = r[8]
        tmp['URL'] = r[9]
        tmp['Enabled'] = r[10]
        tmp['Creator'] = r[11]
        tmp['Create_Date'] = r[12]
        returnData[Answer_NO] = tmp
    response = jsonify(returnData)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response      
@app.route("/getStandandQuestionJson", methods=['GET'])
def getStandandQuestionJson():
    db = request.args.get('db')
    conn = pymysql.connect( host = '10.55.52.98' ,port=33060 ,  user = 'root' ,  passwd = "1234" ,  db = db )     
    cur = conn.cursor() 
        
    returnData = OrderedDict();
    main = OrderedDict();
    key=1
    i=0
    cur.execute('SELECT category_main,count(1) FROM mq_standand_question_new where Question<>'' group by category_main')
    for r in cur :
        tmp = OrderedDict()
        tmp['key'] = key
        tmp['text'] = r[0]
        tmp['isGroup'] = True
        tmp['category'] = "OfGroups"
        returnData[i] = tmp
        key = key + 1
        main[key] = r[0]
        i = i + 1
    
    #for i in range(0,len(main)):
        
     
    response = jsonify(returnData)
    response.headers.add('Access-Control-Allow-Origin', '*')
    return response      
if __name__ == '__main__':    
    #app.run(host='0.0.0.0', port=81)
    run_simple('127.0.0.1', 81, app)        
        
        

 * Running on http://127.0.0.1:81/ (Press CTRL+C to quit)
127.0.0.1 - - [08/Apr/2020 11:00:26] "OPTIONS /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:00:26] "GET /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:25:57] "OPTIONS /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:25:57] "GET /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:32:35] "OPTIONS /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:32:35] "GET /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:33:51] "OPTIONS /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:33:51] "GET /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:34:05] "OPTIONS /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 11:34:05] "GET /getStandandQuestion?db=mq HTTP/1.1" 200 -
127.0.0.1 - - [08/Apr/2020 12:42:05] "OPTIONS /getStandandQuestion?db=mq

In [10]:
# 建立問題與答案的json

import pymysql
import datetime
import time
import json 
from collections import OrderedDict
#db = request.args.get('db')
db = 'mq'
conn = pymysql.connect( host = '10.55.52.98' ,port=33060 ,  user = 'root' ,  passwd = "1234" ,  db = db )     
cur = conn.cursor() 

returnData = {};
main = {};
rd = []
rdA = {}
Gkey=1
i=0

# OfGroups
cur.execute("SELECT category_main,count(1) FROM mq_standand_question_new where Question<>'' group by category_main")
for r in cur :
    tmp = {}
    tmp['key'] = Gkey
    tmp['text'] = r[0]
    tmp['isGroup'] = True
    tmp['category'] = "OfGroups"
    rd.append(tmp)    
    main[Gkey] = r[0]
    Gkey = Gkey + 1
    i = i + 1

# OfNodes    

for Gkey in main: 
    #print(Gkey,' ',main[Gkey])
    category_main = main[Gkey] 
    Nkey = 1
    category_subString = ''
    cur.execute("SELECT category_sub,count(1) FROM mq_standand_question_new where Question<>'' and category_main=%s group by category_sub",(category_main))
    for r in cur :
        category_sub = r[0]
        tmp = {}
        tmp['key'] = Gkey*100+Nkey
        tmp['text'] = str(Nkey)+'.'+category_sub
        tmp['isGroup'] = True
        tmp['category'] = "OfNodes"
        tmp['group'] = Gkey
        rd.append(tmp)
        Nkey = Nkey + 1
    
# text
cur1 = conn.cursor() 
for Gkey in main:     
    category_main = main[Gkey] 
    Nkey = 1    
    cur.execute("SELECT category_sub,count(1) FROM mq_standand_question_new where Question<>'' and category_main=%s group by category_sub",(category_main))
    for r in cur :
        Q = 1
        category_sub = r[0]
        #print(Gkey,' ',main[Gkey],'  ',category_sub)
        #print('  ',category_sub)
        cur1.execute("SELECT Question,Answer,Answer_NO FROM mq_standand_question_new where Question<>'' and category_main=%s and category_sub=%s order by Answer_NO",(category_main,category_sub))
        for r1 in cur1 :
            tmp = {}
            tmpA = {}
            questionKey = Gkey*10000+Nkey*100+Q
            tmp['key'] = questionKey
            tmp['text'] = str(Q)+'.'+r1[0]
            tmp['group'] = Gkey*100+Nkey
            rdA[str(questionKey)] = r1[1]
            rd.append(tmp)
            #{"text":"1.文件定期Review", "group":11, "key":111},
            #print('    Answer_NO : ',r1[2] )
            #print('    Q',Q,'. ',r1[0] )
            #print('    A',Q,'. ',r1[1])
            Q = Q + 1    
        Nkey = Nkey + 1    

In [1]:
import site; site.getsitepackages()

['C:\\Users\\jacob.liang\\AppData\\Local\\Continuum\\anaconda3',
 'C:\\Users\\jacob.liang\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages']

In [11]:
rdA

{'10101': 'TII： TV(電視)、IT (Monitor螢幕、NB筆記型電腦) 、IAVM  \n\n【IAVM 】Industrial (工控)、Amusements (娛樂)、Vehicles (車用)、Medical (醫療)',
 '10102': 'RMA (Returned Material for Authorization)\n返品(客退品及招回品)處理單位',
 '10103': 'MD\n指中小BU (Tablet 平板 /MP手機  /Auto 後車 / CE消費性電子產品/CELL外賣)\n註: CELL外賣係指出貨型態為LCD對組後的中大板玻璃',
 '10104': 'MKT/PRM：Product Marketing 產品行銷',
 '10105': 'BOM：Bill Of Material 材料清單',
 '10106': 'BTC：Bipane technology center 面板技術中心',
 '10107': 'MAC：Mobile / Automotive & Avionics / Consummer \n 手機 / 車用&航太 / 消費性',
 '10108': 'Optical Engineering 光學工程',
 '10109': 'mono產品\n在與多產品當中，僅有Medical 產品使用黑白高解析螢幕，故mono產品長久來就變成Medical team number中的習慣用語，並未定義在文件當中',
 '10110': '醫療用產品\n在Medical team number中討論產品時常會以解析度來代表，如：5M Mono、3M color',
 '10111': 'ORT：On-going Reliability Test 持續性可靠度測試\nOQC 的 ORT實驗項目有: 常溫點燈, 高溫點燈, 高溫高濕, 低溫測試',
 '10112': 'PAD\nProduct Analysis Department 產品解析部',
 '10113': 'CPI\nCustomer Project & Intelligence Management Platform客戶專案管理平台',
 '10114': 'DOA\nDead On Arrival 到貨不良',
 '10115': 'VSS (V

In [7]:
rdA

[{'AE0039': 'TII： TV(電視)、IT (Monitor螢幕、NB筆記型電腦) 、IAVM  \n\n【IAVM 】Industrial (工控)、Amusements (娛樂)、Vehicles (車用)、Medical (醫療)'},
 {'AE0085': 'RMA (Returned Material for Authorization)\n返品(客退品及招回品)處理單位'},
 {'AE0132': 'MD\n指中小BU (Tablet 平板 /MP手機  /Auto 後車 / CE消費性電子產品/CELL外賣)\n註: CELL外賣係指出貨型態為LCD對組後的中大板玻璃'},
 {'AE0166': 'MKT/PRM：Product Marketing 產品行銷'},
 {'AE0168': 'BOM：Bill Of Material 材料清單'},
 {'AE0188': 'BTC：Bipane technology center 面板技術中心'},
 {'AE0191': 'MAC：Mobile / Automotive & Avionics / Consummer \n 手機 / 車用&航太 / 消費性'},
 {'AE0198': 'Optical Engineering 光學工程'},
 {'AE0235': 'mono產品\n在與多產品當中，僅有Medical 產品使用黑白高解析螢幕，故mono產品長久來就變成Medical team number中的習慣用語，並未定義在文件當中'},
 {'AE0236': '醫療用產品\n在Medical team number中討論產品時常會以解析度來代表，如：5M Mono、3M color'},
 {'AE0241': 'ORT：On-going Reliability Test 持續性可靠度測試\nOQC 的 ORT實驗項目有: 常溫點燈, 高溫點燈, 高溫高濕, 低溫測試'},
 {'AE0270': 'PAD\nProduct Analysis Department 產品解析部'},
 {'AE0271': 'CPI\nCustomer Project & Intelligence Management Platform客戶專案管理平台'},
 {'AE0274': 'DOA\

In [17]:
db = 'mq'
conn = pymysql.connect( host = '10.55.52.98' ,port=33060 ,  user = 'root' ,  passwd = "1234" ,  db = db )     
cur = conn.cursor() 
for key in main: 
    print(main[key])
    category_main = main[key]
    cur.execute("SELECT category_sub,count(1) FROM mq_standand_question_new where Question<>'' and category_main=%s group by category_sub",(category_main))
    for r in cur :
        print('  ',r[0])
        

名詞定義
   公司
   品質工作
   定義
品質活動
   NPQM
   OQC
   個人連結
   專案
   指標
   會議
品質系統
   EOQC
   客戶
異常處理
   AERB
   白血球
   防堵
製程生產
   LCD
   LCM
   生產
規格判定
   Mura類
   others defect
   Sample
   亮點類
   其他
   工具/條件
   方法
   暗點類
   線不良
   點不良
